https://www.kaggle.com/c/state-farm-distracted-driver-detection/overview

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Data Preprocessing

In [3]:
data = pd.read_csv('data/driver_imgs_list.csv')

In [4]:
data.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [5]:
# classes = {}
# ct = 0
# for i in data.classname.unique():
#     classes[i] = ct
#     ct += 1
# classes # maybe make it one-hot here ?

In [6]:
X = []
y = []
for i in range(10):
    dir_path =f'data/imgs/train/c{i}'
    files = os.listdir(dir_path)
    for img_name in files[:20]:
        full_path = dir_path + '/' + img_name
        # print(full_path)
        img = np.array(Image.open(full_path))
        # print(img.shape)
        X.append(img)
        y.append(i)

X = torch.tensor(np.array(X)).float().view(-1, 3, 640, 480)
y = torch.tensor(np.array(y)).float()
X.shape, y.shape

(torch.Size([200, 3, 640, 480]), torch.Size([200]))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
class DriverDataset(Dataset):
    def __init__(self, x_, y_):
        self.x_ = x_
        self.y_ = y_
    
    def __len__(self):
        return self.x_.shape[0]
    
    def __getitem__(self, index):
        return self.x_[index], self.y_[index]

In [9]:
train_dataset = DriverDataset(X_train, y_train)
test_dataset = DriverDataset(X_test, y_test)

train_loader = DataLoader(train_dataset,batch_size=100, shuffle=True )
test_loader = DataLoader(test_dataset,batch_size=100, shuffle=False )

## Model

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.l1 = nn.Linear(16*157*117,640)
        self.l2 = nn.Linear(640, 160)
        self.l3 = nn.Linear(160, 10)
    
    def forward(self, x):
        # x -> (n, 3, 640, 480)
        print('cow1')
        out = self.pool(F.relu(self.conv1(x))) # -> (n, 6, 318, 238)
        print('cow2')
        out = self.pool(F.relu(self.conv2(out))) # -> (n, 16, 157, 117)
        print('cow3')
        out = out.view(-1, 16*157*117)
        out = F.relu(self.l1(out))
        out = F.relu(self.l2(out))
        out = self.l3(out)
        return out

model = CNN().to(device)

In [11]:
loss = nn.CrossEntropyLoss().to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [12]:
# model = torch.compile(model)

In [13]:
for epoch in range(10):
    for i, (x_, y_) in enumerate(train_loader):
        x_ = x_.to(float).to(device) # .view(100, 3, 640*480)
        print(x_.dtype)
        
        # forward pass
        y_pred = model(x_)

torch.float64
cow1


RuntimeError: Input type (double) and bias type (float) should be the same

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Data Preprocessing
data = pd.read_csv('data/driver_imgs_list.csv')
data.head()

X = []
y = []
for i in range(10):
    dir_path = f'data/imgs/train/c{i}'
    files = os.listdir(dir_path)
    for img_name in files[:20]:
        full_path = dir_path + '/' + img_name
        img = np.array(Image.open(full_path))
        X.append(img)
        y.append(i)

X = torch.tensor(np.array(X)).float().view(-1, 3, 640, 480)
y = torch.tensor(np.array(y)).long()
X.shape, y.shape

X_train, X_test, y_train, y_test = train_test_split(X, y)

class DriverDataset(Dataset):
    def __init__(self, x_, y_):
        self.x_ = x_
        self.y_ = y_
    
    def __len__(self):
        return self.x_.shape[0]
    
    def __getitem__(self, index):
        return self.x_[index], self.y_[index]

train_dataset = DriverDataset(X_train, y_train)
test_dataset = DriverDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.l1 = nn.Linear(16*157*117, 640)
        self.l2 = nn.Linear(640, 160)
        self.l3 = nn.Linear(160, 10)
    
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(-1, 16*157*117)
        out = F.relu(self.l1(out))
        out = F.relu(self.l2(out))
        out = self.l3(out)
        return out

model = CNN().to(device)
loss = nn.CrossEntropyLoss().to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10):
    for i, (x_, y_) in enumerate(train_loader):
        x_ = x_.to(device).float()  # Ensure input tensor is of type float32
        y_ = y_.to(device)
        
        # Forward pass
        y_pred = model(x_)
        
        # Compute loss
        l = loss(y_pred, y_)
        
        # Backward pass and optimization
        optimiser.zero_grad()
        l.backward()
        optimiser.step()
        
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/10], Step [{i+1}/{len(train_loader)}], Loss: {l.item():.4f}')


RuntimeError: shape '[-1, 293904]' is invalid for input of size 29830400

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Preprocessing
data = pd.read_csv('data/driver_imgs_list.csv')

X = []
y = []
for i in range(10):
    dir_path = f'data/imgs/train/c{i}'
    files = os.listdir(dir_path)
    for img_name in files[:20]:  # Reduced sample size for testing
        full_path = os.path.join(dir_path, img_name)
        img = np.array(Image.open(full_path))
        X.append(img)
        y.append(i)

X = np.array(X)
y = np.array(y)

X = torch.tensor(X, dtype=torch.float32).permute(0, 3, 1, 2)  # Convert to (N, C, H, W)
y = torch.tensor(y, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class DriverDataset(Dataset):
    def __init__(self, x_, y_):
        self.x_ = x_
        self.y_ = y_
    
    def __len__(self):
        return self.x_.shape[0]
    
    def __getitem__(self, index):
        return self.x_[index], self.y_[index]

train_dataset = DriverDataset(X_train, y_train)
test_dataset = DriverDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.l1 = nn.Linear(16 * 157 * 117, 640)
        self.l2 = nn.Linear(640, 160)
        self.l3 = nn.Linear(160, 10)
    
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(-1, 16 * 157 * 117)  # Flatten the output tensor
        out = F.relu(self.l1(out))
        out = F.relu(self.l2(out))
        out = self.l3(out)
        return out

model = CNN().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10):
    for i, (x_, y_) in enumerate(train_loader):
        x_ = x_.to(device)
        y_ = y_.to(device)
        
        # Forward pass
        y_pred = model(x_)
        
        # Compute loss
        loss = loss_fn(y_pred, y_)
        
        # Backward pass and optimization
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/10], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


RuntimeError: shape '[-1, 293904]' is invalid for input of size 29830400